In [ ]:
# 기계 독해는 질문에 대한 답을 문장 또는 문서내에서 찾는 태스크

In [ ]:
!pip install transformers

In [ ]:
import torch
import sys

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 데이터 불러오기

In [ ]:
!mkdir dataset

In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json

--2025-03-19 09:04:11--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   193MB/s    in 0.2s    

2025-03-19 09:04:12 (193 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-03-19 09:04:12--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [ ]:
!mv /content/KorQuAD_v1.0_train.json dataset
!mv /content/KorQuAD_v1.0_dev.json dataset

In [ ]:
import json
from pathlib import Path

In [ ]:
def read_squad(path):
      path = Path(path)
      with open(path, 'rb') as f:
            squad_dict = json.load(f)

      contexts = []
      questions = []
      answers = []
      for group in squad_dict['data']:
            for passage in group['paragraphs']:
                  context = passage['context']
                  for qa in passage['qas']:
                        question = qa['question']
                        for answer in qa['answers']:
                              contexts.append(context)
                              questions.append(question)
                              answers.append(answer)

      return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_squad('dataset/KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('dataset/KorQuAD_v1.0_dev.json')

In [ ]:
print(train_contexts[52471])
print(len(train_contexts[52471]))

사우스런던 브릭스톤 태생 보위는 어릴 적 음악에 대한 관심을 키워왔으며, 결국에는 예술, 음악, 디자인을 배워 전문적인 음악가 경력을 1963년부터 착수했다. 〈Space Oddity〉는 1969년 7월 발표 뒤 영국 음반 차트에서 상위 5위에 오른 그의 첫 싱글이다. 실험 활동을 거친 그는 1972년 음악을 재개, 자신의 글램록 시기 동안 이색적이고 양성적인 제2의 자아인 지기 스타더스트로 활동을 이어나갔다. 성공을 거둔 싱글 〈Starman〉과 전 세계적인 인기를 끈 음반 《The Rise and Fall of Ziggy Stardust and the Spiders from Mars》으로 캐릭터를 내세운 보위는 1975년 "플라스틱 솔"을 캐릭터화시켜 자신을 철저히 바꾼다. 이 행동은 당초 영국에서 그의 열혈팬의 반발을 샀으나 미국에서는 싱글 〈Fame〉과 음반 《Young Americans》을 통해 처음으로 메이저한 성공을 거두게 된다. 1976년 보위는 컬트 영화 《지구에 떨어진 사나이》에 출연하고 음반 《Station to Station》을 발표한다. 이듬해에는 일렉트릭 음악을 접목한 음반 《Low》 (1977)을 발표하면서 음악적 예상을 깨뜨렸다. 이 음반은 브라이언 이노와의 세 번의 협업 중 첫 번째로 이는 이후 "베를린 삼부작"으로 일컬어진다. 뒤를 이어 발표된 《"Heroes"》 (1977)와 《Lodger》 (1979)는 영국 차트 상위 5위에 진입, 지속적인 극찬을 받았다.
734


In [ ]:
print(train_questions[52471])

보위가 1977년 일렉트릭 음악을 접목하여 발표한 음반은?


In [ ]:
print(train_answers[52471])

{'text': 'Low', 'answer_start': 568}


In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        # 모델 학습을 위해 정답 데이터를 만듬
        # 정답 데이터는 start음절과 end음절로 구성
        # 모델은 전체 토큰 중 start token과 end token을 찾아내는 것을 목표로 학습
      gold_text = answer['text']
      start_idx = answer['answer_start']
      end_idx = start_idx + len(gold_text)

      # sometimes squad answers are off by a character or two – fix this
      # 실제 본문에서 해당 음절 번호로 잘라냈을 때, 정답과 같은지 검사해서 start, end를 보정
      # '이순신은 조선 중기의 무신이다' -> '이순신' -> start: 0, end: 4
      if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
      elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1 # When the gold label is off by one character
      elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2 # When the gold label is off by two characters

    return answers

In [ ]:
train_answers = add_end_idx(train_answers, train_contexts)
val_answers = add_end_idx(val_answers, val_contexts)

In [ ]:
print(train_answers[0])
print(val_answers[0])

{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}
{'text': '1989년 2월 15일', 'answer_start': 0, 'answer_end': 12}


# BERT 모델

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer

In [ ]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# 토크나이저의 char_to_token 함수를 호출하면 음절 인덱스를 토큰 인덱스로 바꿔줄 수 있음
# 그리고 시작 위치 또는 끝 위치가 문장의 최대 길이를 넘어섰다면 잘라주는 작업을 진행
# 이 과정을 truncation이라고 부름

def add_token_positions(encodings, answers):
      start_positions = []
      end_positions = []

      # 음절 인덱스를 토큰 인덱스와 mapping
      for i in range(len(answers)):

        # tokenizer의 char_to_token 함수를 호출하면 음절 숫자를 token index로 바꿔줄수 있음
            start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
            end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

            # 아래 부분은 truncation을 위한 과정입니다.
            if start_positions[-1] is None:
                  start_positions[-1] = tokenizer.model_max_length

            if end_positions[-1] is None:
                  end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] + 1)

            # 추가된 예외 처리, 예를들어서 tokenizer와 model input의 max_length가 512인데,
            # start와 end position이 600과 610 이면 둘다 max_length로 변경해야함.
            if start_positions[-1] is None or start_positions[-1] > tokenizer.model_max_length:
                  start_positions[-1] = tokenizer.model_max_length

            if end_positions[-1] is None or end_positions[-1] > tokenizer.model_max_length:
                  end_positions[-1] = tokenizer.model_max_length

      encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

      return encodings

In [ ]:
train_encodings = add_token_positions(train_encodings, train_answers)
val_encodings = add_token_positions(val_encodings, val_answers)

In [ ]:
print(train_encodings.keys())  # 포함된 키 확인
print({key: len(val) for key, val in train_encodings.items()})  # 각 키의 데이터 길이 출력


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])
{'input_ids': 60407, 'token_type_ids': 60407, 'attention_mask': 60407, 'start_positions': 60407, 'end_positions': 60407}


In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
      return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)


In [ ]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

# 기계독해 학습

In [ ]:
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments

model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
      output_dir='./results', # output directory
      num_train_epochs=1, # total number of training epochs
      per_device_train_batch_size=8, # batch size per device during training
      per_device_eval_batch_size=64, # batch size for evaluation
      logging_dir='./logs', # directory for storing logs
      logging_steps=100,
      learning_rate=3e-5,
      save_total_limit=5
)

In [ ]:
model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Step,Training Loss
100,2.782800
200,1.374700
300,1.108800
400,1.063300
500,0.972900
600,0.928700
700,0.896000
800,0.954200
900,0.930500
1000,0.950200


TrainOutput(global_step=7551, training_loss=0.7286454901570848, metrics={'train_runtime': 7346.3241, 'train_samples_per_second': 8.223, 'train_steps_per_second': 1.028, 'total_flos': 1.5784153184151552e+16, 'train_loss': 0.7286454901570848, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/1자연어처리/기계독해")

# 검증

In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [ ]:
context = r"""
이순신(李舜臣, 1545년 4월 28일 ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다.
본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)였으며, 한성 출신이었다.
문반 가문 출신으로 1576년(선조 9년) 무과(武科)에 급제하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라좌도 수군절도사를 거쳐 정헌대부 삼도수군통제사에 이르렀다.
"""

print(nlp(question="이순신이 태어난 날짜는?", context=context))
print(nlp(question="이순신의 본관은?", context=context))
print(nlp(question="이순신의 시호는?", context=context))
print(nlp(question="이순신의 고향은?", context=context))
print(nlp(question="이순신의 마지막 직책은?", context=context))

{'score': 0.039796944707632065, 'start': 106, 'end': 114, 'answer': '한성 출신이었다'}
{'score': 0.8122280240058899, 'start': 72, 'end': 75, 'answer': '덕수('}
{'score': 0.1598239243030548, 'start': 95, 'end': 98, 'answer': '충무('}
{'score': 0.22856706380844116, 'start': 72, 'end': 75, 'answer': '덕수('}
{'score': 0.02668646350502968, 'start': 214, 'end': 222, 'answer': '삼도수군통제사에'}
